In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
from matplotlib.pyplot import rc_context
from scipy import spatial
import anndata as ad
import matplotlib.pyplot as plt
from scipy.stats import zscore

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

In [ ]:
results_file = r'C:\Users\Jessie\Desktop\PFC-DOI-scFlare\RNAseqAnalysis\results.h5ad'  # the file that will store the analysis results
adata = sc.read_10x_mtx(
    r'C:/Users/Jessie/Desktop/PFC-DOI-scFlare/RNAseqAnalysis',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)                              # write a cache file for faster subsequent reading

adata.var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`

## Pre-processing

In [ ]:
sc.pp.filter_cells(adata, min_genes=600)  #cells that contain less than 600 genes
sc.pp.filter_genes(adata, min_cells=2) #genes that are in less than 2 cells
adata.var['mt'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

In [ ]:
#Remove outliers
adata = adata[adata.obs.n_genes_by_counts < 6000, :]
adata = adata[adata.obs.total_counts < 40000, :]
adata = adata[adata.obs.pct_counts_mt < 20, :]


In [ ]:
# Remove sex/activity dependent genes
non_mito_genes_list = [name for name in adata.var_names if not name.startswith('mt-')]
adata = adata[:, non_mito_genes_list]
rmv_list=['Trf','Plp1','Mog','Mobp',"Mfge8","Mbp","Hbb-bs","H2-DMb2","Fos","Jun","Junb","Egr1","Xist","Tsix","Eif2s3y","Uty","Kdm5d"]
non_rmv = [name for name in adata.var_names if name not in rmv_list]
adata = adata[:, non_rmv]

In [ ]:
#Normalize
sc.pp.normalize_total(adata, target_sum= None)
sc.pp.log1p(adata)

## Clustering

In [ ]:
#Labeling highly variable genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pl.highly_variable_genes(adata)

In [ ]:
adata.raw = adata
adata = adata[:, adata.var.highly_variable]
#Regress out effects of total counts per cell and the percentage of mitochondrial genes expressed. 

sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
#Scale the data to unit variance.
sc.pp.scale(adata, max_value=10)

In [ ]:
#Remove the tTa and TRE for clustering; create new object where they are left in 
flicre= adata[:,['tTA','TREmcherry']]
flicre_list=['tTA', 'TREmcherry']
non_flicre=[name for name in adata.var_names if name not in flicre_list]
adata_fli=adata.copy()
adata=adata[:, non_flicre]

### PCA 

In [ ]:
#PCA for dimentionality reduction
sc.tl.pca(adata, svd_solver='arpack')
sc.pl.pca_variance_ratio(adata, n_pcs=50,log=True)
adata.write(results_file)

In [ ]:
#compute neighborhood graph
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=26)
sc.tl.umap(adata)
sc.pl.umap(adata)

In [ ]:
#Cluster and plot the neighborhood graph
sc.tl.leiden(adata, resolution=.1)
sc.pl.umap(adata, color='leiden')

In [ ]:
#Round 1: Remove outliers 
adata.obs=adata.obs.assign(row_number=range(len(adata.obs)))
adata_fli.obs=adata_fli.obs.assign(row_number=range(len(adata_fli.obs)))

x=pd.DataFrame(adata.obsm['X_umap'][0:,0])
y=pd.DataFrame(adata.obsm['X_umap'][0:,1])

num_clusters=adata.obs['leiden'].nunique()
comx=np.array([])
comy=np.array([])
del_outliers_round1=np.array([])

for c in range(0,int(num_clusters)):
    curr_clust=adata.obs[adata.obs['leiden'].isin([str(c)])]['row_number']
    
        # scale to min and max
    xmin=np.min(x.iloc[curr_clust])
    xmax=np.max(x.iloc[curr_clust])
    ymin=np.min(y.iloc[curr_clust])
    ymax=np.max(y.iloc[curr_clust])
        
    curr_x=np.array((x.iloc[curr_clust]-xmin)/(xmax-xmin))
    curr_y=np.array((y.iloc[curr_clust]-ymin)/(ymax-ymin))
    
    comx=np.append(comx,np.median(curr_x))
    comy=np.append(comy,np.median(curr_y))
                                                  
    dist=np.array([])
    for d in range(0,len(curr_x)):
        dist=np.append(dist,spatial.distance.euclidean([comx[c],comy[c]],[float(curr_x[d]),float(curr_y[d])]))
    outliers=np.where(dist>0.5)[0]
    idx=curr_clust[outliers]
    del_outliers_round1=np.append(del_outliers_round1,np.asarray(idx)).astype(int)
    adata=adata[~adata.obs['row_number'].isin(del_outliers_round1)]
    adata_fli=adata_fli[~adata_fli.obs['row_number'].isin(del_outliers_round1)]

In [ ]:
adata.obs=adata.obs.assign(row_number=range(len(adata.obs)))
adata_fli.obs=adata_fli.obs.assign(row_number=range(len(adata_fli.obs)))

In [ ]:
#Round 2: Remove outliers
x=pd.DataFrame(adata.obsm['X_umap'][0:,0])
y=pd.DataFrame(adata.obsm['X_umap'][0:,1])

num_clusters=max(adata.obs['leiden'])
comx=np.array([])
comy=np.array([])
del_outliers_round2=np.array([])
del_barcodes_round2=np.array([])
for c in range(0,int(num_clusters)):
    curr_clust=adata.obs[adata.obs['leiden'].isin([str(c)])]['row_number']
    
        # scale to min and max
    xmin=np.min(x.iloc[curr_clust])
    xmax=np.max(x.iloc[curr_clust])
    ymin=np.min(y.iloc[curr_clust])
    ymax=np.max(y.iloc[curr_clust])
        
    curr_x=np.array((x.iloc[curr_clust]-xmin)/(xmax-xmin))
    curr_y=np.array((y.iloc[curr_clust]-ymin)/(ymax-ymin))
    
    comx=np.append(comx,np.median(curr_x))
    comy=np.append(comy,np.median(curr_y))
                                                  
    dist=np.array([])
    for d in range(0,len(curr_x)):
        dist=np.append(dist,spatial.distance.euclidean([comx[c],comy[c]],[float(curr_x[d]),float(curr_y[d])]))
    outliers=np.where(dist>0.5)[0]
    idx=curr_clust[outliers]
    del_outliers_round2=np.append(del_outliers_round2,np.asarray(idx)).astype(int)
    adata=adata[~adata.obs['row_number'].isin(del_outliers_round2)]
    adata_fli=adata_fli[~adata_fli.obs['row_number'].isin(del_outliers_round2)]

In [ ]:
adata.obs=adata.obs.assign(row_number=range(len(adata.obs)))
adata_fli.obs=adata_fli.obs.assign(row_number=range(len(adata_fli.obs)))

In [ ]:
#Plot new UMAP- Fig S6A
sc.tl.leiden(adata, resolution=.1)
sc.pl.umap(adata, color='leiden')

In [ ]:
#Fnd marker genes
sc.tl.rank_genes_groups(adata, 'leiden', method='t-test')
sc.pl.rank_genes_groups(adata, n_genes=10, sharey=False)

In [ ]:
#Export genes separated by cluster- DataS3
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
cell_genes= pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names','logfoldchanges', 'pvals_adj']})
cell_genes.to_csv('genes_allcells.csv')

In [ ]:
#Annotate cluster 
cluster2annotation = {
     '0': '0Neuron- Exc1',
     '1': '1Neuron- Exc2',
     '2': '2Unk',
     '3': '3Oligodencrocyte',
     '4': '4Microglia',
     '5': '5Neuron- Exc3',
     '6': '6Astrocytes',
     '7': '7Neuron- Inh1',
     '8': '8Neuron- Exc4',
     '9': '9Oligodendrocyte2',
     '10': '10Neuron- Inh2',
     '11': '11Neuron- Exc5',
     '12': '12Ukn2',
}

adata.obs['cell type'] = adata.obs['leiden'].map(cluster2annotation).astype('category')
sc.pl.umap(adata, color='cell type', legend_loc='on data',
           frameon=False, legend_fontsize=10, legend_fontoutline=2)

In [ ]:
adata.write(results_file)

In [ ]:
#Copy barcodes into array with TTA and TRE genes  still present
barcodes=adata.obs.index
barcodes_fli=adata_fli.obs.index
adata.obs['barcodes']=barcodes
adata_fli.obs['barcodes']=barcodes_fli

## Neurons

In [ ]:
#Isolate neurons
neurcol=['0','1','5','7','8','10','11']
neurons= adata[adata.obs['leiden'].isin(neurcol)].copy()
neurons_fli=adata_fli[adata.obs['leiden'].isin(neurcol)].copy()

In [ ]:
# Recluster neurons 
#PCA
sc.tl.pca(neurons, svd_solver='arpack')
sc.pl.pca_variance_ratio(neurons, n_pcs=50, log=True)

In [ ]:
sc.pp.neighbors(neurons, n_neighbors=10, n_pcs=23)
sc.tl.umap(neurons)
sc.pl.umap(neurons)

In [ ]:
#Plot neuronal UMAP-- Fig3B
sc.tl.leiden(neurons, resolution=.2)
sc.pl.umap(neurons, color='leiden')

In [ ]:
#Fnd marker genes
sc.tl.rank_genes_groups(neurons, 'leiden', method='t-test')
sc.pl.rank_genes_groups(neurons, n_genes=10, sharey=False)

In [ ]:
result = neurons.uns['rank_genes_groups']
groups = result['names'].dtype.names
neuron_genes= pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names','logfoldchanges', 'pvals_adj']})
neuron_genes.to_csv('genes_neurons.csv')

In [ ]:
newneurcol=['0','1','2','3','4','5','6','7','8']

neurons_fli=neurons_fli[neurons.obs['leiden'].isin(newneurcol)].copy()
neurons= neurons[neurons.obs['leiden'].isin(newneurcol)].copy()
neurons_fli.obs['leiden']=neurons.obs['leiden']

In [ ]:
cluster2annotation = {
     '0': 'Exc1',
     '1': 'Exc2',
     '2': 'Exc3',
     '3': 'Exc4',
     '4': 'Inh1',
     '5': 'Exc5',
     '6': 'Inh2',
     '7': 'Exc6',
     '8': 'Exc7'

}
neurons.obs['cell type'] = neurons.obs['leiden'].map(cluster2annotation).astype('category')
sc.pl.umap(neurons, color='cell type', legend_loc='on data',
           frameon=False, legend_fontsize=10, legend_fontoutline=2)

In [ ]:
##Multigene matrix plot-- Fig3C
clust_genes_dict = {
    'Exc1': ['Cd44'],
    'Exc2': ['Foxp2', 'Syt6', 'Sla'],
    'Exc3': ['Otof', 'Cux2'],
    'Exc4': ['Npr3'],
    'Inh1': ['Sst','Pvalb'],
    'Exc5': ['Tshz2'],
    'Inh2': ['Vip'],
    'Exc6': ['Pld5'],
    'Exc7': ['Cdh18', 'Tle4']
}


sc.pl.matrixplot(neurons, clust_genes_dict, 'cell type', cmap='Blues', standard_scale='var', show=False)
plt.savefig("neuron_marker_heatmapOtherGenes.pdf")

In [ ]:
##UMAP gene plot-- Fig3D/E
sc.pl.umap(neurons, color=['leiden','tTA'], vmax=0.5)
sc.pl.umap(neurons, color=['leiden','TREmcherry'], vmax=1)

In [ ]:
#Normalized tTA/TREmCherry expression-- Fig3F/G
salineNfli_f1=neurons_fli[neurons_fli.obs['barcodes'].str.contains('-1|-2')].copy()
doiNfli_f1=neurons_fli[neurons_fli.obs['barcodes'].str.contains('-3|-4')].copy()

ax=sc.pl.violin(salineNfli_f1, keys='TREmcherry',use_raw=False, show=False)
ax.set_ylim(bottom=-2,top=10.5)
plt.savefig("tresal_violin.pdf")
ax=sc.pl.violin(doiNfli_f1, keys='TREmcherry',use_raw=False,show=False)
ax.set_ylim(bottom=-2,top=10.5)
plt.savefig("tredoi_violin.pdf")

ax=sc.pl.violin(salineNfli_f1, keys='tTA',use_raw=False, show=False)
ax.set_ylim(bottom=-2,top=10.5)
plt.savefig("tresal_violin.pdf")
ax=sc.pl.violin(doiNfli_f1, keys='tTA',use_raw=False,show=False)
ax.set_ylim(bottom=-2,top=10.5)
plt.savefig("tredoi_violin.pdf")

In [ ]:
#Quantify #Normalized gene expression of tTA and TRE-Fig 3H
genedf = sc.get.obs_df(
         neurons_fli,
         keys=["leiden", 'TREmcherry','tTA','barcodes'])
genedf['z_scoreTRE'] = zscore(genedf['TREmcherry'])
genedf['z_scoretTA'] = zscore(genedf['tTA'])
   
#Classify barcodes SalineDOI
salineNfli_f1=genedf[genedf['barcodes'].str.contains('-1|-2')].copy()
doiNfli_f1=genedf[genedf['barcodes'].str.contains('-3|-4')].copy()

#Gene(tTA and TRE) by cluster
clusters = genedf['leiden'].cat.categories

#Repeat for each cluster
saline=salineNfli_f1[salineNfli_f1['leiden'].isin(['0'])]
doi=doiNfli_f1[doiNfli_f1['leiden'].isin(['0'])]
saline.to_csv("C0S_Zratio.csv")
doi.to_csv("C0D_Zratio.csv")

In [ ]:
#Dotplot of TREmCherry -- Fig3I
sc.pl.dotplot(salineNfli_f1, ['TREmcherry'], 'leiden')
sc.pl.dotplot(doiNfli_f1, ['TREmcherry'], 'leiden')
#Dotplot of tTA -- Fig S6C
sc.pl.dotplot(salineNfli_f1, ['tTA'], 'leiden')
sc.pl.dotplot(doiNfli_f1, ['tTA'], 'leiden')

In [ ]:
#umap of Htrta/Htr2c -- Fig4A/D
sc.pl.umap(neurons, color=['leiden','Htr2a'], vmax=3)
sc.pl.umap(neurons, color=['leiden','Htr2c'], vmax=3)


In [ ]:
#Expression of 5HT2A/5HT2C-- Fig4B,E
salineN=neurons[neurons.obs['barcodes'].str.contains('-1|-2')].copy()
doiN=neurons[neurons.obs['barcodes'].str.contains('-3|-4')].copy()

salineNfli_f1=neurons_fli[neurons_fli.obs['barcodes'].str.contains('-1|-2')].copy()
doiNfli_f1=neurons_fli[neurons_fli.obs['barcodes'].str.contains('-3|-4')].copy()

gene_ids = ['Htr2c','Htr2a']

obsSf1 = salineNfli_f1.raw[:,gene_ids].X.toarray()
obsDf1 = doiNfli_f1.raw[:,gene_ids].X.toarray()
obsSf1 = pd.DataFrame(obsSf1,columns=['Htr2c','Htr2a']),index=salineNfli_f1.obs['leiden'])
obsDf1 = pd.DataFrame(obsDf1,columns=['Htr2c','Htr2a']),index=doiNfli_f1.obs['leiden'])

saline=obsSf1.loc['0'] #change for each cluster
doi=obsDf1.loc['0'] #change for each cluster

In [ ]:
#Dotplot of Htrta/Htr2c -- Fig4C/F
sc.pl.dotplot(neurons, ['Htr2a'], 'leiden')
sc.pl.dotplot(neurons, ['Htr2c'], 'leiden')